# OCO-3 Snapshot Area Map (SAM) Exploration

WWhile OCO-2 and OCO-3 share the same spectrometer design, OCO-3 introduces a key innovation: the Pointing Mirror Assembly (PMA). This PMA enables rapid reorienting of the instrument's field of view, allowing data collection over contiguous 80 km × 80 km areas called Snapshot Area Maps (SAMs). These SAMs provide spatially dense SIF measurements that capture fine-scale heterogeneity in vegetation photosynthesis. The OCO-3 science team maintains an archive of SAM acquisitions at [https://ocov3.jpl.nasa.gov/sams/](https://ocov3.jpl.nasa.gov/sams/)

In this notebook, we will analyze a time series of SAMs centered on the University of Michigan Biological Station (UMBS) in northern Michigan, USA. This site sits within a deciduous broadleaf forest ecosystem with >60% vegetation cover, interspersed with small agricultural areas and rural communities. By examining SAMs acquired across different seasons, we will:

1. Quantify the seasonal dynamics of SIF in this temperate forest ecosystem. Additionally, data has been prepared for you in the homework exercise that covers an evergreen needleleaf forest.
2. Compare OCO-3 SIF observations with ground-based gross primary productivity (GPP) measurements from the [AmeriFlux eddy covariance tower (US-UMB)](https://ameriflux.lbl.gov/sites/siteinfo/US-UMB) 
3. Explore the spatial patterns of photosynthetic activity across the study region

This exercise will demonstrate how SAMs can bridge the gap between the high temporal resolution of tower measurements and the high spatial coverage of satellite observations.

In [ ]:
import contextily as ctx
from cartopy import crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from datetime import datetime
import json
from matplotlib.collections import PatchCollection
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sys


# Add src directory containing helper code to sys.path
sys.path.append(os.path.abspath("../src"))

from pysif import GesDiscDownloader, plot_samples

## I. Retrieving SAM Data

The [SAM webpage](https://ocov3.jpl.nasa.gov/sams/index.php) mentioned previously is a great place to start for finding specific sites and observations. In this exercise, we will be looking at the `sif_University_of_Michigan_USA` site, but you can pan through the map on the webpage to find a different area of interest on your own. The homework exercise covers the `ecostress_us_me2` site as an additional example.

<div align="center">
  <img src="images/sif_uofm_usa_map.png" alt="sif_University_of_Michigan_USA SAM site map pin">
</div>

If you put the site name and the site type ("SIF_High", which can be found by clicking on the map pin for the site as can be seen in the figure above) into the search parameters, a list of all SAM observations at that site will be pulled up.

* Site name: `sif_University_of_Michigan_USA`
* Site type: `SIF_High`

These search results can tell us which dates we want to retrieve SIF granules for, and then filter those granules to only include soundings from the SAM mode, which is `Metadata_MeasurementMode` 3 in the granule. To perform the retrieval and plotting, we can use the same code from the first exercise (1_exploration.ipynb). For the best possible plot, let's choose a SAM snapshot with a non-zero number of soundings. A good example of this occurred on August 11th, 2020:

<div align="center">
  <img src="images/sam_search_result.png" alt="SAM mode search result">
</div>

In [ ]:
dl = GesDiscDownloader()

dataset = "OCO3_L2_Lite_SIF.11r"
data_date = datetime(2020, 8, 11)
granule = dl.get_granule_by_date(dataset, data_date)

Now that we have the granule, we can filter the data to just SAM soundings. You can find more information about OCO-3's measurement modes in the [User's Guide](https://docserver.gesdisc.eosdis.nasa.gov/public/project/OCO/OCO2_v11.2_OCO3_v11_SIF_Data_Users_Guide.pdf), see Section 4.7 "The Metadata Group". Measurement Mode 3 is referred to as an AreaMap -- this is the SAM mode we are interested in.

In [ ]:
# The sam_extent is determined manually by inspecting the official SAM mode plot
# If you choose a different date or tower site, adjust these values accordingly!
sam_extent = [-86.8, -83.9, 44.7, 46.5]

# Get the measurement mode of the data first so we can filter to just use SAM mode soundings
# Additionally, we will remove low quality soundings
meas_mode = np.array(granule["Metadata_MeasurementMode"].data[:])
qual_flag = np.array(granule["Quality_Flag"].data[:])
sam_mask  = (meas_mode == 3) & ((qual_flag == 0) | (qual_flag == 1))

all_lat = granule["Latitude"].data[:]
all_lon = granule["Longitude"].data[:]
all_sif = granule["Daily_SIF_757nm"].data[:]

lat = all_lat[sam_mask]
lon = all_lon[sam_mask]
sif = all_sif[sam_mask]

# Using the plot_samples function from the first notebook gives us a good first look at this data
# before further processing. The YlGn colormap is used to make it easier to compare the data with
# the official plot.
plot_samples(
    sif, lat, lon,
    vmin=0.21,
    vmax=0.64,
    cmap="YlGn",
    point_size=15,
    fig_size=(8, 8),
    extents=sam_extent,
    title=f"OCO-3 SAM mode LtSIF 757 nm ({data_date.strftime('%Y-%m-%d')})",
    label=r"Daily SIF 757 nm (W/$\mathrm{m}^2$/sr/μm)"
)

### Comparison with official plot
If you've been following along on the OCO-3 website, you may have noticed that the science team provides pre-made plots of a variety of measurements for each SAM acquisition. Let's take a look at one of these plots for the OCO-3 LtSIF 757nm variable:

<div style="text-align:center">
  <img style="width:640px" src="images/OCO3_Lite_B11074Ar_r02_sif_757nm_20200811_7215_sif_umb.png" alt="Official OCO-3 SAM mode plot over UMBS tower site for August 11, 2020">
</div>

If the previous code ran successfully for you, you should be able to see that we are at least looking at the same data but with some superficial differences in presentation. In the next section, we will refine our plot to look like the official version. The most important component of this will be to adjust each sounding to use its proper footprint extents. We will also add a marker to the plot to indicate the location of the tower.

In [ ]:
# Be sure to specify the location of the tower as a (longitude, latitude)
tower_lonlat = (-84.7138, 45.5598)
vmin = 0.21
vmax = 0.64


def plot_sam(
        granule,
        extent: list[float],
        tower_site: tuple[float, float] = (-1, -1),
        vmin: float = -0.05,
        vmax: float = 0.55,
) -> None:
    # Perform the same filtering step as in the previous code block
    meas_mode = np.array(granule["Metadata_MeasurementMode"].data[:])
    qual_flag = np.array(granule["Quality_Flag"].data[:])
    sam_mask  = (meas_mode == 3) & ((qual_flag == 0) | (qual_flag == 1))
    
    # Instead of using latitude and longitude directly, we will now apply the
    # footprint vertices when plotting SIF samples
    all_footlat = granule["Geolocation_footprint_latitude_vertices"].data[:]
    all_footlon = granule["Geolocation_footprint_longitude_vertices"].data[:]
    footprint_lat = all_footlat[sam_mask][:]
    footprint_lon = all_footlon[sam_mask][:]

    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.GOOGLE_MERCATOR)

    ax.set_extent(extent, crs=ccrs.PlateCarree())

    # Add a marker with the tower location to the plot. These coordinates can be found on the Ameriflux site
    if tower_site != (-1, -1):
        ax.plot(tower_site[0], tower_site[1], marker='*', color='red', markersize=10, transform=ccrs.Geodetic())

    patches_list = []
    for i in range(len(sif)):
        polygon = patches.Polygon(np.column_stack([footprint_lon[i], footprint_lat[i]]))
        patches_list.append(polygon)

    pcol = PatchCollection(
        patches_list,
        cmap="YlGn",
        transform=ccrs.PlateCarree()
    )

    pcol.set_array(np.array(sif))
    pcol.set_clim(vmin, vmax)
    sif_layer = ax.add_collection(pcol)

    # Zoom level 9 strikes a balance between detail in the map base layer and
    # execution time when creating the plot. If you have a slow internet connection,
    # try decreasing the zoom value to 7 or 8.
    ctx.add_basemap(
        ax,
        source=ctx.providers.Esri.WorldImagery,
        crs=ccrs.GOOGLE_MERCATOR,
        zoom=9
    )

    # Add gridlines over the map. Note that we can use the default LATITUDE_FORMATTER and
    # LONGITUDE_FORMATTER from cartopy to make our lives easier
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color="white", alpha=0.7, linestyle="-")
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {"size": 14, "color": "black"}
    gl.ylabel_style = {"size": 14, "color": "black"}

    plt.title(f"OCO-3 SAM mode LtSIF 757 nm ({data_date.strftime("%Y-%m-%d")})", fontsize=16)
    cbar = plt.colorbar(sif_layer, ax=ax, extend="both", pad=0.01)
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label(r"Daily SIF 757 nm [W/$\mathrm{m}^2$/sr/μm]", fontsize=16, rotation=270, labelpad=20)

    plt.tight_layout()
    plt.show()

plot_sam(granule, sam_extent, tower_lonlat, vmin, vmax)


## II. Using Ground-Based Data from an Ameriflux Tower

[Eddy covariance (EC)](https://en.wikipedia.org/wiki/Eddy_covariance) flux towers continuously measure carbon dioxide exchange between ecosystems and the atmosphere. These towers provide high-frequency measurments, typically every 30 minutes, of climatology as well as two key variables related to SIF:

* **Net Ecosystem Exchange (NEE)**: The net CO₂ flux between the ecosystem and atmosphere
* **Gross Primary Production (GPP)**: Total photosynthetic carbon uptake by vegetation.

The observed area extends approximately 1 km around the tower, so despite the low spatial coverage the high temporal resolution enables flux towers to act as a ground truth for satellite observations. The figure below illustrates how tower measurements bridge the scale gap between leaf-level measurements and remote sensing:

<div style="text-align:center">
  <img style="width:640px" src="images/scale.png" alt="A diagram showing the scale of various methods for measuring vegetation primary productivity">
</div>

### Working with Ameriflux Data

To begin, we will plot the GPP data contained in the provided `AMF_US-UMB_FLUXNET_SUBSET_DD_2019-2021.csv` file. This file was obtained using the "Download Data" functionality on the [Ameriflux page for the tower](https://ameriflux.lbl.gov/sites/siteinfo/US-UMB). Downloading data requires an account, so this step has already been done for your convenience. The Ameriflux data is presented at several time scales ranging from yearly to hourly, and for this analysis we are just using the daily data. An abridged version of the daily data is included in the notebooks directory containing just the entries for August 2019 through the end of 2021. A guide to the filename convention is included to help with understanding the characteristics of the data:

> `AMF_US-UMB_FLUXNET_SUBSET_DD_2019-2021.csv`
> * `AMF` = Ameriflux network data
> * `US-UMB` = tower site code, United States University of Michigan Biological (Station)
> * `FLUXNET` = fluxnet data, as opposed to BADM, BASE, or other datasets. Note that BASE datasets do not contain GPP.
> * `SUBSET` = this file contains only a subset of the full set of measurements available for the tower. The available measurements are documented on the webpage.
> * `DD` = Daily time cadence
> * `2019-2021` = File has been edited to contain only relevant dates. This is a subset of the original 2007-2021 date range for this site.

**In the following code block, we will compare two methods for retrieving GPP, the Daytime (DT) and Nighttime (NT) partionining Variable Ustar Threshold (VUT) algorithms. We will also plot the OCO-3 SIF values from the same period using a spatial averaging technique described in the next section.**

For more information on the theoretical basis of these GPP retrievals, please read the following papers:

* [Pastorello et al., 2020](https://doi.org/10.1038/s41597-020-0534-3): description of the FLUXNET pipeline
* [Lasslop et al., 2010](https://doi.org/10.1111/j.1365-2486.2009.02041.x): daytime partitioning algorithm for GPP/NEE
* [Reichstein et al., 2005](https://doi.org/10.1111/j.1365-2486.2005.001002.x): nighttime partitioning algorithm for GPP

In [ ]:
# File paths: If you want to study a different tower like the us_me2 example, modify the file paths here
gpp_file = "AMF_US-UMB_FLUXNET_SUBSET_DD_2019-2021.csv"
sif_file = "us_umb_oco3_sif.json"

# gpp_file = "AMF_US-Me2_FLUXNET_SUBSET_DD_2019-2022.csv"
# sif_file = "us_me2_oco3_sif.json"

# Load SIF observation data
def load_sif_data(json_file):
    """Load SIF observation data from JSON file, excluding dates with SIF = 0"""
    with open(json_file, "r") as f:
        data = json.load(f)
    
    # Extract dates and SIF values where SIF > 0
    sif_data = [(pd.to_datetime(item["date"]), item["sif"]) 
                for item in data["dates"] if item["sif"] != 0]
    
    sif_dates = [item[0] for item in sif_data]
    sif_values = [item[1] for item in sif_data]
    
    return sif_dates, sif_values

# Read the GPP file
tower_name = gpp_file.split("_")[1]
date_range = gpp_file.split("_")[5].strip(".csv")

# strip metadata rows that start with #
df = pd.read_csv(gpp_file, comment='#')

# FLUXNET daily timestamps are yyyymmdd as an int/str.
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], format='%Y%m%d')
df = df.set_index('TIMESTAMP')

# Get the two "REF" GPP columns
cols = [
    "GPP_NT_VUT_REF",
    "GPP_DT_VUT_REF"
]
gpp = df[cols]

# Load SIF observation data, if available
try:
    sif_dates, sif_values = load_sif_data(sif_file)
    
    # Create the plot with dual y-axes
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    # Plot GPP time series on primary y-axis
    gpp.plot(ax=ax1, linewidth=1.5, alpha=0.8)
    ax1.set_xlabel("Date", fontsize=12)
    ax1.set_ylabel("Tower GPP [gC/m²/day]", fontsize=12, color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid(True, alpha=0.3)
    
    # Create secondary y-axis for SIF
    ax2 = ax1.twinx()
    ax2.scatter(sif_dates, sif_values, color='red', marker='o', s=50, 
               alpha=0.7, edgecolors='darkred', linewidth=0.5, 
               label='SIF Observations', zorder=5)
    ax2.set_ylabel(r'OCO-3 SIF [W/m²/sr/μm]', fontsize=12, color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    
    plt.title(f"Daily GPP (NT vs DT VUT) and SIF Observations for {tower_name}, {date_range}", fontsize=14)
    
    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, 
              loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)
    
    plt.tight_layout()
    plt.show()
    
except FileNotFoundError as e:
    print(f"Error: Could not find SIF file - {e}")
    # Fallback: plot without SIF markers
    plt.figure(figsize=(12, 6))
    gpp.plot(ax=plt.gca())
    plt.xlabel("Date")
    plt.ylabel("Tower GPP [gC/m²/day]")

    plt.title(f"Daily GPP (NT vs DT VUT) for {tower_name}, {date_range}")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error loading SIF data: {e}")

### What can we notice about this plot?

The two GPP partitioning methods (DT and NT) show strong agreement throughout the observation period at US-UMB. This consistency indicates robust flux measurements and stable ecosystem behavior. Note that algorithm divergence can occur at some sites, and you'll observe this phenomenon in the US-Me2 homework dataset around summer 2022.

Another key observation is that the data reveal distinct phenological patterns characteristic of deciduous broadleaf forests:

* **Growing season (May–October)**: Peak GPP values reaching ~15 gC m⁻² day⁻¹
* **Dormant season (November–April)**: Near-zero GPP during leafless periods
* **Transitions**: Rapid changes during spring leaf-out and autumn senescence

This seasonality contrasts with evergreen forests, which exhibit some measurable GPP and SIF at a reduced level throughout winter months. 

The SIF observations from OCO-3, denoted by the red markers, are reasonably well correlated with the GPP data even despite being different measurements acquired by different instruments at vastly different resolutions. This suggests that the SIF signal we are getting from OCO-3 is effectively capturing ecosystem-scale photosynthesis and that both measurements respond similarly to environmental factors.

**Question: How do you expect an ecological disruption would affect the ability to discern a SIF-GPP relationship between OCO-3 data and tower data? Try replacing the files in the previous code block and the next with the provided CSV and JSON for the US-Me2 site. What can you observe about that data that looks different from the US-UMB site?**

## III. Comparison of OCO-3 SIF with Ameriflux Tower GPP Data

In this section, we will compare daily averaged SIF from OCO-3 against tower GPP data from the US-UMB site. Ultimately, we will create a daily time-scale plot comparing available data from the two datasets over the period from August 2019 (the beginning of the OCO-3 mission) through the end of 2021 (the end of the Ameriflux data for this tower as of the writing of this training). This plot will look similar to figure 3b from [Pierrat et al., 2022](https://doi.org/10.1029/2021JG006588). 

If you wish to explore the data from that paper and other tower-based SIF and GPP measurements further, you can find the associated dataset here: 
* [https://doi.org/10.5281/zenodo.10048770](https://doi.org/10.5281/zenodo.10048770): Zenodo data store with tower SIF at various sites
* [https://climatesciences.jpl.nasa.gov/sif/download-data/tower/](https://climatesciences.jpl.nasa.gov/sif/download-data/tower/): A curated list of towers that have SIF data. Remember that because OCO-3 has only been operating since 2019, not all tower sites will have data records that overlap with OCO-3.

### Pre-processing

Several pre-processing steps have already been taken for you. First, you will notice there is an ancillary file called "us_umb_oco3_dates.json" in the same directory as this notebook. This JSON file was created by studying all available data dates across the OCO-3 mission from August 2019 - December 2021 to determine which samples lie within a 0.5-degree radius around the tower site. This search process was performed using the code in the Appendix notebook, so we will just discuss it conceptually here. By looping through each date in the time range of interest, we can search for (longitude, latitude) coordinate pairs that lie within the defined bounding box: in this case, $\pm 0.25^{\circ}$ around the tower coordinates. In the next part of the pre-processing, we extract the relevant data from each discovered granule and take the mean of all 757nm SIF data with a Quality Flag value of 0 (best) or 1 (good). The spatial means are stored in a separate JSON file called "us_umb_oco3_sif.json" that will be used as the basis for our daily SIF values in the comparison plot.

In [ ]:
# File paths
gpp_file = "AMF_US-UMB_FLUXNET_SUBSET_DD_2019-2021.csv"
sif_file = "us_umb_oco3_sif.json"

# gpp_file = "AMF_US-Me2_FLUXNET_SUBSET_DD_2019-2022.csv"
# sif_file = "us_me2_oco3_sif.json"

# Change this to use a different GPP column, e.g., GPP_DT_VUT_REF
# For US-UMB, this has little effect on the regression
gpp_column = "GPP_NT_VUT_REF"


def load_sif_data(json_file):
    """Load SIF data from JSON file"""
    with open(json_file, "r") as f:
        data = json.load(f)
    
    # Convert to DataFrame
    df = pd.DataFrame(data["dates"])
    df["date"] = pd.to_datetime(df["date"])
    return df

def load_gpp_data(csv_file, gpp_column="GPP_NT_VUT_REF"):
    """Load GPP data from CSV file"""
    df = pd.read_csv(csv_file)
    
    # Convert TIMESTAMP to datetime
    df["date"] = pd.to_datetime(df["TIMESTAMP"], format="%Y%m%d")
    
    # Select relevant columns
    df = df[["date", gpp_column]].copy()
    df.rename(columns={gpp_column: "gpp"}, inplace=True)
    
    return df

def add_temporal_features(data):
    """Add year and season columns to the data"""
    data = data.copy()
    
    # Extract year
    data["year"] = data["date"].dt.year
    
    # Extract month and map to seasons
    month = data["date"].dt.month
    data["season"] = month.map({
        12: "winter", 1: "winter", 2: "winter",
        3: "spring", 4: "spring", 5: "spring",
        6: "summer", 7: "summer", 8: "summer",
        9: "autumn", 10: "autumn", 11: "autumn"
    })
    
    return data

def merge_and_clean_data(sif_df, gpp_df):
    """Merge SIF and GPP data on date and remove invalid values"""
    # Merge on date
    merged = pd.merge(sif_df, gpp_df, on="date", how="inner")
    
    # Remove rows with NaN or negative values
    merged = merged.dropna()
    
    # Remove rows where GPP is the fill value, -9999
    merged = merged[merged["gpp"] != -9999]
    merged = merged[merged["sif"] != 0]
    
    # Add temporal features
    merged = add_temporal_features(merged)
    
    return merged

def create_scatter_plot(data, gpp_column_name="GPP_NT_VUT_REF", mark_years=False, print_stats=False):
    """Create scatter plot with linear regression, seasonal colors, and yearly markers"""
    # Extract x and y values
    x = data["sif"].values.reshape(-1, 1)
    y = data["gpp"].values

    # Fit linear regression
    reg = LinearRegression()
    reg.fit(x, y)
    y_pred = reg.predict(x)
    r2 = r2_score(y, y_pred)
    
    # Create figure
    plt.figure(figsize=(14, 8))
    
    # Define color mapping for seasons
    season_colors = {
        "winter": "blue",
        "spring": "green",
        "summer": "gold",
        "autumn": "red"
    }
    
    if mark_years:
        year_markers = {
            2019: "o",  # circle
            2020: "s",  # square
            2021: "^",  # triangle up
            2022: "D"   # diamond
        }
        title_addon = " | Markers: Years"
    else:
        year_markers = {y: "o" for y in range(2019, 2023)}
        title_addon = ""
    
    # Create scatter plot with seasonal colors and yearly markers
    for year in sorted(data["year"].unique()):
        for season in ["winter", "spring", "summer", "autumn"]:
            # Filter data for this year and season
            mask = (data["year"] == year) & (data["season"] == season)
            subset = data[mask]
            
            if len(subset) > 0:
                plt.scatter(
                    subset["sif"], 
                    subset["gpp"],
                    c=season_colors[season],
                    marker=year_markers[year],
                    s=60,
                    alpha=0.7,
                    edgecolors="black",
                    linewidth=0.5,
                    label=f"{year} {season}" if len(plt.gca().get_legend_handles_labels()[0]) < 16 else ""
                )
    
    # Plot regression line
    x_sorted = np.sort(data["sif"].values)
    y_pred_sorted = reg.predict(x_sorted.reshape(-1, 1))
    plt.plot(x_sorted, y_pred_sorted, color="black", linewidth=2, linestyle="--", alpha=0.8)
    
    # Add R² value to plot
    plt.text(0.05, 0.95, f"R² = {r2:.3f}", 
             transform=plt.gca().transAxes, 
             fontsize=12, 
             verticalalignment="top",
             bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))
    
    # Labels and title
    plt.xlabel(r"OCO-3 SIF [W/$\mathrm{m}^2$/sr/μm]", fontsize=12)
    plt.ylabel(f"Tower GPP ({gpp_column_name}) " + r"[gC/$\mathrm{m}^2$/day]", fontsize=12)
    plt.title(f"Solar-Induced Fluorescence (SIF) vs Gross Primary Productivity (GPP)\nColors: Seasons{title_addon}", fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Season legend (colors)
    season_handles = [plt.scatter([], [], c=color, s=60, marker='o', edgecolors="black", linewidth=0.5) 
                     for season, color in season_colors.items()]
    season_labels = [season.capitalize() for season in season_colors.keys()]

    legend1 = plt.legend(season_handles, season_labels,
                         loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=4)
    #                    loc='upper left', bbox_to_anchor=(1, 1))
    
    if mark_years:
        # Year legend (markers)
        year_handles = [plt.scatter([], [], c='gray', marker=marker, s=60, edgecolors="black", linewidth=0.5) 
                    for year, marker in year_markers.items()]
        year_labels = [str(year) for year in year_markers.keys()]

        legend2 = plt.legend(year_handles, year_labels, title="Years", 
                            loc='upper left', bbox_to_anchor=(1.02, 0.7))
        
        # Add the first legend back (matplotlib removes it when creating the second)
        plt.gca().add_artist(legend1)
    
    if print_stats:
        # Print statistics
        print(f"Number of matched data points: {len(data)}")
        print(f"Linear regression equation: y = {reg.coef_[0]:.3f}x + {reg.intercept_:.3f}")
        print(f"R² value: {r2:.3f}")
        print(f"Slope: {reg.coef_[0]:.3f}")
        print(f"Intercept: {reg.intercept_:.3f}")
    
    plt.tight_layout()
    plt.show()

try:
    sif_data = load_sif_data(sif_file)
    
    gpp_data = load_gpp_data(gpp_file, gpp_column)
    
    merged_data = merge_and_clean_data(sif_data, gpp_data)
    
    # Set mark_years to True if you would like different years of data
    # to receive different markers on the plot
    # Set print_stats to True if you would like a printout of info
    # about the linear regression
    create_scatter_plot(
        merged_data,
        gpp_column,
        mark_years=False,
        print_stats=False
    ) 
except FileNotFoundError as e:
    print(f"Error: File not found - {e}")
except Exception as e:
    print(f"Error: {e}")


### What can we notice about this plot?

Our R² result of 0.73 suggests that these variables are well correlated, and is only a little less than the R² reported from comparing daily tower SIF to daily GPP in Pierrat, et al. 2022. We can improve the correlation further by filtering the data on biome type as will be discussed later.

The plot also can aid with identifying potential biases in the data. The three points far to the right side of the regression line indicate days where SIF was high but GPP was lower than might be expected. We can look at the SAM plot for one of these days to see if there are any pitfalls to our spatial aggregation technique that might introduce issues in the comparison.

The summer data point furthest to the right corresponds to the date 2021-06-02, which can be found by searching through the source JSON file.

In [ ]:
# Follow the same procedure as in Section 1 but for 2021-05-28
data_date_2 = datetime(2021, 6, 2)
# Extent may need to be modified slightly to investigate other SAM mode acquisitions
sam_extent_2 = [-85.9, -82.8, 44.9, 47]
granule = dl.get_granule_by_date(dataset, data_date_2)

vmin = 0.21
vmax = 0.82

# This function is defined in Section I, so re-run those cells if this doesn't work
plot_sam(granule, sam_extent_2, tower_lonlat, vmin, vmax)

In the above plot, we can see that while there are some large SIF values near the tower, some are also quite low. Despite the high data availability in this example, there are still gaps caused by clouds and low quality data. SAM observations, despite providing improved spatial coverage compared with the more common nadir and glint modes, must be spatially aggregated to account for cases when a SIF sample may not fall over the precise location of the tower and to reduce the effects of surface anisotropy. One challenge of spatial aggregation is that SIF can vary widely throughout a small area and even between adjacent samples. In this example, there are many potential reasons for the higher than expected SIF average, but one area to consider is the region of agricultural fields to the east of the tower with high SIF values. It is often useful to examine satellite imagery of your study region to determine the unique characteristics of both the site and its surrounding context.

Depending on the context of the tower site in your own research, it is worth considering the following additional techiques:

* **Reducing the spatial aggregation window**, which will reduce the number of samples considered but potentially exclude non-representative biomes. In the appendix, you can test this by reducing the value of the `tolerance` variable to 0.125
* **Filtering data by biome type**, this can allow you to consider a wider spatial window while still taking into account varying biomes. In the appendix, you can test this by uncommenting this portion of the filtering step: `cond = (qual_flag < 2) & ((igbp_type == 4) + (igbp_type == 5))`
* **Correlating the local solar time of the SAM data to hourly tower data**. Each data point in a granule contains a UTC time associated with the overflight time. As SAMs are acquired over a period of several minutes, this can be compared with the nearest hour in the tower GPP data. 